In [11]:
import json
import pandas as pd
from kafka import KafkaProducer
from time import time

In [12]:
server = 'localhost:9092'

In [ ]:
def json_serializer(obj):
    if isinstance(obj, pd.Timestamp):
        return obj.isoformat()
    raise TypeError("Type not serializable")

In [ ]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=lambda v: json.dumps(v, default=json_serializer).encode('utf-8')
)


In [13]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [14]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

In [15]:
producer.bootstrap_connected()

True

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

In [ ]:
!gzip -d green_tripdata_2019-10.csv.gz

In [16]:
df = pd.read_csv('green_tripdata_2019-10.csv')

/tmp/ipykernel_493303/2156106363.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('green_tripdata_2019-10.csv')


In [7]:
df.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1,112,196,1,5.88,18.0,0.50,0.5,0.00,0.0,NaN,0.3,19.30,2,1,0.0
1,1,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1,43,263,1,0.80,5.0,3.25,0.5,0.00,0.0,NaN,0.3,9.05,2,1,0.0
2,1,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1,255,228,2,7.50,21.5,0.50,0.5,0.00,0.0,NaN,0.3,22.80,2,1,0.0
3,1,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1,181,181,1,0.90,5.5,0.50,0.5,0.00,0.0,NaN,0.3,6.80,2,1,0.0
4,2,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1,97,188,1,2.52,10.0,0.50,0.5,2.26,0.0,NaN,0.3,13.56,1,1,0.0
5,2,2019-10-01 00:35:01,2019-10-01 00:43:40,N,1,65,49,1,1.47,8.0,0.50,0.5,1.86,0.0,NaN,0.3,11.16,1,1,0.0
6,1,2019-10-01 00:28:09,2019-10-01 00:30:49,N,1,7,179,1,0.60,4.0,0.50,0.5,1.00,0.0,NaN,0.3,6.30,1,1,0.0
7,2,2019-10-01 00:28:26,2019-10-01 00:32:01,N,1,41,74,1,0.56,4.5,0.50,0.5,0.00,0.0,NaN,0.3,5.80,2,1,0.0
8,2,2019-10-01 00:14:01,2019-10-01 00:26:16,N,1,255,49,1,2.42,10.5,0.50,0.5,0.00,0.0,NaN,0.3,11.80,2,1,0.0
9,1,2019-10-01 00:03:03,2019-10-01 00:17:13,Y,1,130,131,1,3.40,13.0,0.50,0.5,2.85,0.0,NaN,0.3,17.15,1,1,0.0


In [17]:
df = df[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   lpep_pickup_datetime   476386 non-null  object 
 1   lpep_dropoff_datetime  476386 non-null  object 
 2   PULocationID           476386 non-null  int64  
 3   DOLocationID           476386 non-null  int64  
 4   passenger_count        387007 non-null  float64
 5   trip_distance          476386 non-null  float64
 6   tip_amount             476386 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 25.4+ MB


In [ ]:
#df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

In [ ]:
#df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [19]:
t0 = time()

for record in df.to_dict(orient="records"):
    producer.send('green-trips', value=record)

producer.flush()

t1 = time()
took = t1 - t0

In [20]:
print(took)

49.229318141937256
